# NomisWeb

It was a massive PITA tracking down these codes...

- TYPE150: 2021 output areas within England and Wales
- TYPE151: 2021 super output areas - lower layer within England and Wales
- TYPE152: 2021 super output areas - middle layer within England and Wales
- TYPE153: 2022 wards within England and Wales
- TYPE154: 2022 local authorities: districts within England and Wales
- TYPE155: 2022 local authorities: counties within England and Wales
- TYPE168: 2021 national parks within England and Wales
- TYPE423: local authorities: county / unitary (as of April 2023) within England and Wales
- TYPE424: local authorities: district / unitary (as of April 2023) within England and Wales
- TYPE459: local enterprise partnerships (as of April 2021) within England and Wales
- TYPE480: regions within England and Wales


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

from nomisweb import FieldMetadata, TableMetadata, build_geog_query, fetch, fetch_table
from utils import extract_crime_data

In [3]:
table_name = "NM_2041_1"  # random 2021 census table - age, ethnicity, sex

top_level_geogs = FieldMetadata(**fetch(f"dataset/{table_name}/geography.def.sdmx.json"))
top_level_geogs.to_dataframe()

,Description,TypeName,TypeCode,GeogCode
NomisCode,,,,
2092957703,England and Wales,countries,499,K04000001
2092957699,England,countries,499,E92000001
2092957700,Wales,countries,499,W92000004


In [4]:
# list the available geography types
ew_geog_types = FieldMetadata(**fetch(f"dataset/{table_name}/geography/2092957703.def.sdmx.json"))
ew_geog_types.to_dataframe()

,Description,TypeName,TypeCode,GeogCode,IsAbstractCode,ParentCode,ChildCount
NomisCode,,,,,,,
2092957703,England and Wales,countries,499,K04000001,NaN,NaN,NaN
2092957703TYPE150,2021 output areas within England and Wales,2021 output areas,150,NaN,True,2.092958e+09,188880.0
2092957703TYPE151,2021 super output areas - lower layer within E...,2021 super output areas - lower layer,151,NaN,True,2.092958e+09,35672.0
2092957703TYPE152,2021 super output areas - middle layer within ...,2021 super output areas - middle layer,152,NaN,True,2.092958e+09,7264.0
2092957703TYPE153,2022 wards within England and Wales,2022 wards,153,NaN,True,2.092958e+09,7638.0
2092957703TYPE154,2022 local authorities: districts within Engla...,2022 local authorities: districts,154,NaN,True,2.092958e+09,331.0
2092957703TYPE155,2022 local authorities: counties within Englan...,2022 local authorities: counties,155,NaN,True,2.092958e+09,174.0
2092957703TYPE168,2021 national parks within England and Wales,2021 national parks,168,NaN,True,2.092958e+09,13.0
2092957703TYPE423,local authorities: county / unitary (as of Apr...,local authorities: county / unitary (as of Apr...,423,NaN,True,2.092958e+09,175.0


In [5]:
# list specific geographies of a given type
ew_lads = FieldMetadata(**fetch(f"dataset/{table_name}/geography/2092957703TYPE154.def.sdmx.json"))
ew_lads.to_dataframe()

,Description,TypeName,TypeCode,GeogCode
NomisCode,,,,
645922819,Hartlepool,2022 local authorities: districts,154,E06000001
645922820,Middlesbrough,2022 local authorities: districts,154,E06000002
645922822,Redcar and Cleveland,2022 local authorities: districts,154,E06000003
645922823,Stockton-on-Tees,2022 local authorities: districts,154,E06000004
645922817,Darlington,2022 local authorities: districts,154,E06000005
...,...,...,...,...
645923145,Torfaen,2022 local authorities: districts,154,W06000020
645923146,Monmouthshire,2022 local authorities: districts,154,W06000021
645923147,Newport,2022 local authorities: districts,154,W06000022


In [6]:
# codelist endpoint will give every supported geography for a given table (cached as its large)
geog_df = Path("./data/census2021geographies.parquet")
if not geog_df.exists():
    # seems like codelist endpoints dont like api keys
    all_geogs = FieldMetadata(**fetch("codelist/CL_2041_1_GEOGRAPHY.def.sdmx.json")).to_dataframe()
    all_geogs.to_parquet(geog_df)
else:
    all_geogs = pd.read_parquet(geog_df)

all_geogs  # .TypeName.value_counts()

,NomisCode,TypeName,TypeCode
GeogCode,,,
E00060274,629202434,2021 output areas,150
E00060275,629202435,2021 output areas,150
E00060276,629202436,2021 output areas,150
E00060277,629202437,2021 output areas,150
E00060279,629202439,2021 output areas,150
...,...,...,...
E12000009,2013265929,regions,480
W92000004,2013265930,regions,480
K04000001,2092957703,countries,499


## Get table metadata

In [7]:
metadata = TableMetadata(**fetch(f"dataset/{table_name}.def.sdmx.json"))

# table info
for a in metadata.structure.keyfamilies.keyfamily[0].annotations.annotation:
    print(a.annotationtitle, a.annotationtext)

Status Current (being actively updated)
Keywords Ethnic group
Units Persons
contenttype/sources census_2021_ts
contenttype/geoglevel oa2021,lsoa2021,msoa2021,la2021,ward2021,oa,msoa,la
SubDescription All usual residents
Mnemonic c2021ts021
FirstReleased 2022-11-29 09:30:00
LastUpdated 2022-11-29 11:30:00
LastRevised 2022-11-29 11:30:00
MetadataTitle0 About this dataset
MetadataText0 This dataset provides Census 2021 estimates that classify usual residents in England and Wales by ethnic group. The estimates are as at Census Day, 21 March 2021.
   
   National Park data are created by plotting unique properties as identified by their Unique Property Reference Number or postcodes into National Park boundaries current at December 2022. This differs from the OA best fit methodology used for other geographic level data.
MetadataTitle1 Protecting personal data
MetadataText1 Sometimes we need to make changes to data if it is possible to identify individuals. This is known as statistical disclo

In [8]:
fields = {
    a.conceptref: {"codelist": a.codelist} for a in metadata.structure.keyfamilies.keyfamily[0].components.dimension
}
fields

{'GEOGRAPHY': {'codelist': 'CL_2041_1_GEOGRAPHY'},
 'C2021_ETH_20': {'codelist': 'CL_2041_1_C2021_ETH_20'},
 'MEASURES': {'codelist': 'CL_2041_1_MEASURES'},
 'FREQ': {'codelist': 'CL_2041_1_FREQ'}}

In [9]:
# get metadata for each field
for field_name, metadata in fields.items():
    if field_name == "GEOGRAPHY":
        continue

    metadata["values"] = FieldMetadata(**fetch(f"codelist/{metadata['codelist']}.def.sdmx.json")).to_dataframe()

In [10]:
fields["C2021_ETH_20"]["values"]

,Description,ChildCount,FirstChildCode,FirstChildTypeCode,IsTotal,IsDefault,TypeName,TypeCode,Level,isDerived,DisplayName
NomisCode,,,,,,,,,,,
0,Total: All usual residents,5.0,1001.0,1000000.0,True,True,Ethnic group,1000000,0,NaN,NaN
1001,"Asian, Asian British or Asian Welsh",5.0,12.0,1000000.0,NaN,NaN,Ethnic group,1000000,1,True,NaN
12,"Asian, Asian British or Asian Welsh: Bangladeshi",NaN,NaN,NaN,NaN,NaN,Ethnic group,1000000,2,NaN,Bangladeshi
13,"Asian, Asian British or Asian Welsh: Chinese",NaN,NaN,NaN,NaN,NaN,Ethnic group,1000000,2,NaN,Chinese
10,"Asian, Asian British or Asian Welsh: Indian",NaN,NaN,NaN,NaN,NaN,Ethnic group,1000000,2,NaN,Indian
11,"Asian, Asian British or Asian Welsh: Pakistani",NaN,NaN,NaN,NaN,NaN,Ethnic group,1000000,2,NaN,Pakistani
14,"Asian, Asian British or Asian Welsh: Other Asian",NaN,NaN,NaN,NaN,NaN,Ethnic group,1000000,2,NaN,Other Asian
1002,"Black, Black British, Black Welsh, Caribbean o...",3.0,16.0,1000000.0,NaN,NaN,Ethnic group,1000000,1,True,NaN
16,"Black, Black British, Black Welsh, Caribbean o...",NaN,NaN,NaN,NaN,NaN,Ethnic group,1000000,2,NaN,African


## Get some crime data and the LSOAs they occur in 

In [12]:
crime_data = extract_crime_data("west-yorkshire")
crime_data

,Month,Reported by,Falls within,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,geometry
Crime ID,,,,,,,,,,
de5777487ac2e8e65f9e3dcca6324000603d7b9f194cc876a7307c7c73178bf9,2022-05,West Yorkshire Police,West Yorkshire Police,On or near Huddersfield Road,E01007426,Barnsley 027D,Public order,Unable to prosecute suspect,NaN,POINT (421953.015 406383.969)
78c826e8faaef52a848f768caf94b636c5cac1c434b8718da613c2b1fa5e3ee7,2022-05,West Yorkshire Police,West Yorkshire Police,On or near Croft House Fold,E01010646,Bradford 001A,Other theft,Unable to prosecute suspect,NaN,POINT (408213.025 449655.983)
5d757bc36423a00b912d71b5084f4cf71b968c81cd354e5d1262f2838f420858,2022-05,West Yorkshire Police,West Yorkshire Police,On or near North Street,E01010646,Bradford 001A,Public order,Unable to prosecute suspect,NaN,POINT (408339.003 449760.012)
1dc7a8dbaf4fe640e89b8f310579cbc9c8e74002a42afec93982fcfeee29e377,2022-05,West Yorkshire Police,West Yorkshire Police,On or near Church Street,E01010646,Bradford 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,POINT (408359.006 449659.021)
9512e607ed96486619d9b23702cd3126112730e864d400d4978d1d605b489fc2,2022-05,West Yorkshire Police,West Yorkshire Police,On or near Parking Area,E01010646,Bradford 001A,Violence and sexual offences,Investigation complete; no suspect identified,NaN,POINT (407832.994 449794.972)
...,...,...,...,...,...,...,...,...,...,...
a8cf6a7848c41fcda8fedb73c7c70408da4a7d2bf41da0a44e40a61b9c51895b,2025-04,West Yorkshire Police,West Yorkshire Police,On or near Grove Way,E01011872,Wakefield 045D,Violence and sexual offences,Under investigation,NaN,POINT (444686.999 410596.051)
c1547cb03e3a3e6e0d574bec4af8df310d17a6458a4ecb89c5cbe93d180ad5ed,2025-04,West Yorkshire Police,West Yorkshire Police,On or near Radford Park Avenue,E01011872,Wakefield 045D,Violence and sexual offences,Under investigation,NaN,POINT (444593.011 410321.009)
3186127efc7e7b1de2da82f2deefc0924266bd518eff0ce6f37fdb3301aa6f5c,2025-04,West Yorkshire Police,West Yorkshire Police,On or near Clifford Road,E01011872,Wakefield 045D,Violence and sexual offences,Unable to prosecute suspect,NaN,POINT (444662.025 410807.995)


In [13]:
available_lsoas = FieldMetadata(
    **fetch(f"dataset/{table_name}/geography/2092957703TYPE151.def.sdmx.json")
).to_dataframe()["GeogCode"]
lsoas = crime_data["LSOA code"].unique()
# FFS crime data has some 2011 LSOAs
lsoas = np.intersect1d(lsoas, available_lsoas)
lsoas

array(['E01005410', 'E01005414', 'E01005448', ..., 'E01035052',
       'E01035053', 'E01035054'], shape=(1445,), dtype=object)

In [14]:
table_name = "NM_2132_1"
nomis_area_codes = available_lsoas[available_lsoas.isin(lsoas)].index.to_list()

# TODO? select only using codes and use metadata as lookup?
selections = ",".join(
    (
        "GEOGRAPHY_CODE",
        *(f"{field}_NAME" for field in fields if field not in ["GEOGRAPHY", "FREQ", "MEASURES"]),
        "OBS_VALUE",
    )
)

params = {
    "date": "latest",
    "geography": build_geog_query(nomis_area_codes),
    "c2021_eth_20": "1001...1005",
    "c2021_age_6": "1...5",
    "c_sex": "1,2",
    "select": selections,
}

data = fetch_table(table_name, **params)

In [15]:
data  # .C2021_ETH_20_NAME.unique()

,GEOGRAPHY_CODE,C2021_ETH_20_NAME,OBS_VALUE
0,E01005410,"Asian, Asian British or Asian Welsh",1
1,E01005410,"Asian, Asian British or Asian Welsh",1
2,E01005410,"Asian, Asian British or Asian Welsh",0
3,E01005410,"Asian, Asian British or Asian Welsh",0
4,E01005410,"Asian, Asian British or Asian Welsh",4
...,...,...,...
72245,E01035054,Other ethnic group,13
72246,E01035054,Other ethnic group,2
72247,E01035054,Other ethnic group,1
72248,E01035054,Other ethnic group,0


In [16]:
# e.g. compare proportion of Black people in community to stop-and-search incidences

data["is_black"] = data.C2021_ETH_20_NAME.str.contains("Black")
lsoa_totals = data.groupby(["GEOGRAPHY_CODE", "is_black"]).OBS_VALUE.sum().unstack(level="is_black")

proportion = lsoa_totals.apply(lambda r: r[True] / r.sum(), axis=1)
proportion

GEOGRAPHY_CODE
E01005410    0.000668
E01005414    0.001498
E01005448    0.057352
E01005561    0.004954
E01006881    0.010497
               ...   
E01035050    0.090965
E01035051    0.045877
E01035052    0.037216
E01035053    0.016110
E01035054    0.039875
Length: 1445, dtype: float64